# Read README.md

In [ ]:
from selenium.webdriver.chrome.service import Service
import pandas as pd
from selenium import webdriver 
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.chrome.options import Options
import time
import keyboard

In [49]:
options = Options()
options.add_argument('--headless')  # Run Chrome in headless mode
options.add_argument('--disable-gpu')  # Disable GPU
options.add_argument('--no-sandbox')  # Bypass OS security model
options.add_argument('--disable-dev-shm-usage')  # Overcome limited resource problems

In [ ]:
# Load the CSV file into a DataFrame with the first row as the header
df = pd.read_csv('sports_car(uds).csv', usecols=['Car Make', 'Car Model', 'Year'])

# Convert the DataFrame to a list of tuples
search_terms = df.apply(lambda row: f"{row['Car Make']}, {row['Car Model']}, {row['Year']}", axis=1).tolist()
search_index = 0

# Set up Chrome options
options = webdriver.ChromeOptions()
options.add_argument("user-data-dir=C:/Users/hp/AppData/Local/Google/Chrome/User Data")

# Specify the path to the ChromeDriver executable
service = Service(executable_path="E:/New folder (3)/chromedriver.exe")

# Initialize the WebDriver with the specified service and options
driver = webdriver.Chrome(service=service, options=options)

In [ ]:
def search_next_term():
    global search_index
    if search_index < len(search_terms):
        search_term = search_terms[search_index]
        search_index += 1

        # Open Google
        driver.get('https://www.google.com/')
        
        # Wait for the search box to be present
        wait = WebDriverWait(driver, 20)  # Increased timeout to 20 seconds
        search_box = wait.until(EC.presence_of_element_located((By.NAME, 'q')))
        
        # Enter the search term in the search box and press Enter
        search_box.send_keys(search_term)
        search_box.send_keys(Keys.RETURN)
        try:
            # Wait until the search results are loaded
            wait.until(EC.presence_of_element_located((By.ID, 'search')))
        except TimeoutException:
            print(f"Timeout while waiting for search results for term: {search_term}")
        finally:
            # Wait for the backtick key "`" to be pressed before proceeding
            print(f"Press ` to search for the next term: {search_term}")
            while True:
                if keyboard.is_pressed('`'):
                    break
                time.sleep(0.1)
            # Open a new window and search for the next term
            driver.execute_script("window.open('');")
            driver.switch_to.window(driver.window_handles[-1])
            search_next_term()

# Call the function to start the search
search_next_term()

In [ ]:
# Close the WebDriver session
driver.quit()